# LETTER-2

## ('A'-'M' positive class, 'N'-'X' negative class)
### Source:
https://archive.ics.uci.edu/ml/datasets/Letter+Recognition
### Goal: 
Identify black-and-white rectangular pixel displays as one of the 26 capital letters in the English alphabet

# IMPORTS

In [19]:
import numpy as np
import pandas as pd
import requests, zipfile, io
import pickle

from sklearn.metrics import accuracy_score, f1_score, auc, roc_auc_score, roc_curve, make_scorer
from sklearn.model_selection import KFold, train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier

#plots
import seaborn as sns
import matplotlib.pyplot as plt

# Algo 1: LOGISTIC REGRESSION

In [2]:
def log_reg_classifier(X_train, Y_train, X_test, Y_test):
    # define hyperparameters to search through & error metrics 
    C_list = [10**-8,10**-7,10**-6,10**-5,10**-4,10**-3,10**-2,10**-1,10**0,10**1,10**2,10**3,10**4]
    penalty_list = ['l2', 'l1', 'none']
    solver = ['saga'] #this solver works for no penalty, l1 and l2 penalties
    scoring = {'Accuracy': make_scorer(accuracy_score), 'f1': 'f1', 'AUC': 'roc_auc'}
    classifier = LogisticRegression()
        
    # gridsearch 
    lr =  GridSearchCV(classifier, {'C':C_list, 'penalty':penalty_list,'solver':solver},
                                   scoring=scoring,refit=False, cv=5, n_jobs=-1, 
                                   return_train_score=True, verbose=2 ) 
        
    lr.fit(X_train,Y_train)
    results = pd.DataFrame(lr.cv_results_)
        
    # Get each parameter settings that gives best accuracy, F1, and AUC on validation set
    best_AUC = results[results['rank_test_AUC']==1]['params'].values[0]
    best_accuracy = results[results['rank_test_Accuracy']  ==1]['params'].values[0]
    best_F1 = results[results['rank_test_f1']  ==1]['params'].values[0]
        
    # Train 3 models using the 5000 samples and each of the 3 best parameter settings (one model per metric)
    clf_AUC = LogisticRegression(C=best_AUC['C'], solver = best_AUC['solver'],
                             penalty=best_AUC['penalty']).fit(X_train, Y_train)

    clf_accuracy = LogisticRegression(C=best_accuracy['C'], solver = best_accuracy['solver'],
                             penalty=best_accuracy['penalty']).fit(X_train, Y_train)

    clf_F1 = LogisticRegression(C=best_F1['C'], solver = best_F1['solver'],
                                penalty=best_F1['penalty']).fit(X_train, Y_train)
        
    # For average training performance
    train_accuracy = clf_accuracy.score(X_train, Y_train)  # Accuracy  
    train_F1 = f1_score(Y_train, clf_F1.predict(X_train))   # F1
    fpr, tpr, threshold = roc_curve(Y_train, clf_AUC.predict_proba(X_train)[:,1])
    train_auc = auc(fpr, tpr) # AUC
        
    # Find and store accuracy, F1, and AUC of the 3 models from previous line on test set
    test_accuracy = clf_accuracy.score(X_test, Y_test)  # Accuracy
    
    test_F1 = f1_score(Y_test, clf_F1.predict(X_test))   # F1
    
    false_positive_rate, true_positive_rate, thresholds = roc_curve(Y_test, clf_AUC.predict_proba(X_test)[:,1])
    test_auc = auc(false_positive_rate, true_positive_rate) # AUC

    return train_accuracy, train_F1, train_auc, test_accuracy, test_F1, test_auc


# Algo 2: RANDOM FOREST

In [3]:
def rf_classifier(X_train,Y_train,X_test,Y_test):
    # define hyperparameters to search through & error metrics  
    n_estimators = [1024]
    min_samples_split = [1,2,4,6,8,12,16,20]
    scoring = {'Accuracy': make_scorer(accuracy_score), 'f1': 'f1', 'AUC': 'roc_auc'}
    classifier = RandomForestClassifier()

    rf =  GridSearchCV(classifier, {'n_estimators':n_estimators, 'min_samples_split':min_samples_split},
                                   scoring=scoring,refit=False, cv=5, n_jobs=-1, 
                                   return_train_score=True, verbose=2) 

    rf.fit(X_train,Y_train)
    results = pd.DataFrame(rf.cv_results_)
        
    # Get each parameter settings that gives best accuracy, F1, and AUC on validation set
    best_AUC = results[results['rank_test_AUC']==1]['params'].values[0]
    best_accuracy = results[results['rank_test_Accuracy']  ==1]['params'].values[0]
    best_F1 = results[results['rank_test_f1']  ==1]['params'].values[0]

    # Train 3 models using the 5000 samples and each of the 3 best parameter settings (one model per metric)
    clf_AUC = RandomForestClassifier(n_estimators=best_AUC['n_estimators'], 
                              min_samples_split=best_AUC['min_samples_split']).fit(X_train, Y_train)

    clf_accuracy = RandomForestClassifier(n_estimators=best_accuracy['n_estimators'], 
                              min_samples_split=best_accuracy['min_samples_split']).fit(X_train, Y_train)

    clf_F1 = RandomForestClassifier(n_estimators=best_F1['n_estimators'], 
                                  min_samples_split=best_F1['min_samples_split']).fit(X_train, Y_train)

    # For average training performance
    train_accuracy = clf_accuracy.score(X_train, Y_train)  # Accuracy  
    train_F1 = f1_score(Y_train, clf_F1.predict(X_train))   # F1
    fpr, tpr, threshold = roc_curve(Y_train, clf_AUC.predict_proba(X_train)[:,1])
    train_auc = auc(fpr, tpr) # AUC
        
    # Find and store accuracy, F1, and AUC of the 3 models from previous line on test set
    test_accuracy = clf_accuracy.score(X_test, Y_test)  # Accuracy
    
    test_F1 = f1_score(Y_test, clf_F1.predict(X_test))   # F1
    
    false_positive_rate, true_positive_rate, thresholds = roc_curve(Y_test, clf_AUC.predict_proba(X_test)[:,1])
    test_auc = auc(false_positive_rate, true_positive_rate) # AUC

    return train_accuracy, train_F1, train_auc, test_accuracy, test_F1, test_auc
  

# Algo 3: KNN

In [4]:
def knn_classifier(X_train,Y_train,X_test,Y_test):
    # define hyperparameters to search through & error metrics  
    n_neighbors = np.linspace(1, 105, num=27, dtype=int)
    weights = ['uniform', 'distance']
    p = [1,2]
    scoring = {'Accuracy': make_scorer(accuracy_score), 'f1': 'f1', 'AUC': 'roc_auc'}
    classifier = KNeighborsClassifier()

    rf =  GridSearchCV(classifier, {'n_neighbors':n_neighbors, 'weights': weights, 'p':p},
                                   scoring=scoring,refit=False, cv=5, n_jobs=-1, 
                                   return_train_score=True, verbose=2) 

    rf.fit(X_train,Y_train)
    results = pd.DataFrame(rf.cv_results_)
        
    # Get each parameter settings that gives best accuracy, F1, and AUC on validation set
    best_AUC = results[results['rank_test_AUC']==1]['params'].values[0]
    best_accuracy = results[results['rank_test_Accuracy']  ==1]['params'].values[0]
    best_F1 = results[results['rank_test_f1']  ==1]['params'].values[0]

    # Train 3 models using the 5000 samples and each of the 3 best parameter settings (one model per metric)
    clf_AUC = KNeighborsClassifier(n_neighbors=best_AUC['n_neighbors'], 
                             weights=best_AUC['weights'], p = best_AUC['p']).fit(X_train, Y_train)

    clf_accuracy = KNeighborsClassifier(n_neighbors=best_accuracy['n_neighbors'], 
                              weights=best_accuracy['weights']).fit(X_train, Y_train)

    clf_F1 = KNeighborsClassifier(n_neighbors=best_F1['n_neighbors'], 
                                  weights=best_F1['weights'], p = best_F1['p']).fit(X_train, Y_train)

    # For average training performance
    train_accuracy = clf_accuracy.score(X_train, Y_train)  # Accuracy  
    train_F1 = f1_score(Y_train, clf_F1.predict(X_train))   # F1
    fpr, tpr, threshold = roc_curve(Y_train, clf_AUC.predict_proba(X_train)[:,1])
    train_auc = auc(fpr, tpr) # AUC
        
    # Find and store accuracy, F1, and AUC of the 3 models from previous line on test set
    test_accuracy = clf_accuracy.score(X_test, Y_test)  # Accuracy
    
    test_F1 = f1_score(Y_test, clf_F1.predict(X_test))   # F1
    
    false_positive_rate, true_positive_rate, thresholds = roc_curve(Y_test, clf_AUC.predict_proba(X_test)[:,1])
    test_auc = auc(false_positive_rate, true_positive_rate) # AUC

    return train_accuracy, train_F1, train_auc, test_accuracy, test_F1, test_auc  

# DATA

In [29]:
data = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/letter-recognition/letter-recognition.data', header=None)
print(data.shape)
data.info()
data.head()

(20000, 17)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 17 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   0       20000 non-null  object
 1   1       20000 non-null  int64 
 2   2       20000 non-null  int64 
 3   3       20000 non-null  int64 
 4   4       20000 non-null  int64 
 5   5       20000 non-null  int64 
 6   6       20000 non-null  int64 
 7   7       20000 non-null  int64 
 8   8       20000 non-null  int64 
 9   9       20000 non-null  int64 
 10  10      20000 non-null  int64 
 11  11      20000 non-null  int64 
 12  12      20000 non-null  int64 
 13  13      20000 non-null  int64 
 14  14      20000 non-null  int64 
 15  15      20000 non-null  int64 
 16  16      20000 non-null  int64 
dtypes: int64(16), object(1)
memory usage: 2.6+ MB


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
0,T,2,8,3,5,1,8,13,0,6,6,10,8,0,8,0,8
1,I,5,12,3,7,2,10,5,5,4,13,3,9,2,8,4,10
2,D,4,11,6,8,6,10,6,2,6,10,3,7,3,7,3,9
3,N,7,11,6,6,3,5,9,4,6,4,4,10,6,10,2,8
4,G,2,1,3,1,1,8,6,6,6,6,5,9,1,7,5,10


In [30]:
#checking if any observations have missing data
rows_to_drop = data[data.isnull().any(axis=1)] 
print(rows_to_drop)

Empty DataFrame
Columns: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16]
Index: []


In [31]:
# get inputs (X) & targets (y) 
X = data.loc[: , 1:16] #feature inputs: taking colums "1" to "16"
labels = data[0]

#determining +/- classes, A-M = positive class, N-Z = negative class
Y=[]
for i in labels:
    if i in ('A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M'):
        Y += [1]
    else:
        Y += [-1]
        
# add label column to original dataframe (for plot)
data['y'] = Y


In [8]:
# %POZ
print(data['y'].value_counts())

percent_positive = 9940/20000
print(percent_positive)

-1    10060
 1     9940
Name: y, dtype: int64
0.497


### 5 Trials across the three algos 

In [9]:
log_reg_data = []
rf_data = []
knn_data = []

for i in range(5):
    #split data for training and testing
    X_train, X_test, Y_train, Y_test = train_test_split(X,Y, train_size=5000)

        
    #tansform data for Logistic Regression and KNN
    X_train_t = StandardScaler().fit_transform(X_train)    
    X_test_t = StandardScaler().fit_transform(X_test)
    
    log_reg_data += log_reg_classifier(X_train_t, Y_train, X_test_t, Y_test)
    rf_data += rf_classifier(X_train,Y_train, X_test, Y_test)
    knn_data += knn_classifier(X_train_t,Y_train, X_test_t, Y_test)
    
    

Fitting 5 folds for each of 39 candidates, totalling 195 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:    3.5s
[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed:    8.0s
[Parallel(n_jobs=-1)]: Done 195 out of 195 | elapsed:    9.6s finished
/Users/annaleaohalloran/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:1505: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  "Setting penalty='none' will ignore the C and l1_ratio "


Fitting 5 folds for each of 8 candidates, totalling 40 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:  2.4min
[Parallel(n_jobs=-1)]: Done  40 out of  40 | elapsed:  2.7min finished


Fitting 5 folds for each of 108 candidates, totalling 540 fits


...

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  58 tasks      | elapsed:    2.1s
[Parallel(n_jobs=-1)]: Done 188 out of 195 | elapsed:    6.5s remaining:    0.2s
[Parallel(n_jobs=-1)]: Done 195 out of 195 | elapsed:    6.7s finished


Fitting 5 folds for each of 8 candidates, totalling 40 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
/Users/annaleaohalloran/opt/anaconda3/lib/python3.7/site-packages/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:  2.6min
[Parallel(n_jobs=-1)]: Done  40 out of  40 | elapsed:  3.2min finished


Fitting 5 folds for each of 108 candidates, totalling 540 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:   20.9s
[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed:  2.6min
[Parallel(n_jobs=-1)]: Done 357 tasks      | elapsed:  6.1min
[Parallel(n_jobs=-1)]: Done 540 out of 540 | elapsed:  8.5min finished


In [26]:
#Logistic Regression Results
df_log_reg = pd.DataFrame(columns=('Train ACC', 'Train F1','Train AUC', 'Test ACC', 'Test F1', 'Test AUC'))
df_log_reg.loc['LR Letter 2 Trial 1'] = log_reg_data[0:6]
df_log_reg.loc['LR Letter 2 Trial 2'] = log_reg_data[6:12]
df_log_reg.loc['LR Letter 2 Trial 3'] = log_reg_data[12:18]
df_log_reg.loc['LR Letter 2 Trial 4'] = log_reg_data[18:24]
df_log_reg.loc['LR Letter 2 Trial 5'] = log_reg_data[24:32]

#Random Forest Results
df_rf = pd.DataFrame(columns=('Train ACC', 'Train F1','Train AUC', 'Test ACC', 'Test F1', 'Test AUC'))
df_rf.loc['RF Letter 2 Trial 1'] = rf_data[0:6]
df_rf.loc['RF Letter 2 Trial 2'] = rf_data[6:12]
df_rf.loc['RF Letter 2 Trial 3'] = rf_data[12:18]
df_rf.loc['RF Letter 2 Trial 4'] = rf_data[18:24]
df_rf.loc['RF Letter 2 Trial 5'] = rf_data[24:32]


#KNN Results
df_knn = pd.DataFrame(columns=('Train ACC', 'Train F1','Train AUC', 'Test ACC', 'Test F1', 'Test AUC'))
df_knn.loc['KNN Letter 2 Trial 1'] = knn_data[0:6]
df_knn.loc['KNN Letter 2 Trial 2'] = knn_data[6:12]
df_knn.loc['KNN Letter 2 Trial 3'] = knn_data[12:18]
df_knn.loc['KNN Letter 2 Trial 4'] = knn_data[18:24]
df_knn.loc['KNN Letter 2 Trial 5'] = knn_data[24:32]

print('Logistic Regression Results')
display(df_log_reg)
print(df_log_reg.mean())

print('\n\nRandom Forest Results')
display(df_rf)
print(df_rf.mean())

print('\n\nKNN Results')
display(df_knn)
print(df_knn.mean())


Logistic Regression Results


,Train ACC,Train F1,Train AUC,Test ACC,Test F1,Test AUC
LR Letter 2 Trial 1,0.7290,0.732161,0.818212,0.723733,0.725061,0.811628
LR Letter 2 Trial 2,0.7326,0.731364,0.820606,0.721200,0.719743,0.810016
LR Letter 2 Trial 3,0.7252,0.728994,0.812997,0.726933,0.730100,0.813332
LR Letter 2 Trial 4,0.7254,0.725015,0.811110,0.720733,0.720155,0.811495
LR Letter 2 Trial 5,0.7332,0.744836,0.816511,0.727600,0.735115,0.812139


Train ACC    0.729080
Train F1     0.732474
Train AUC    0.815887
Test ACC     0.724040
Test F1      0.726035
Test AUC     0.811722
dtype: float64


Random Forest Results


,Train ACC,Train F1,Train AUC,Test ACC,Test F1,Test AUC
RF Letter 2 Trial 1,1.0,1.0,1.0,0.944800,0.943772,0.989866
RF Letter 2 Trial 2,1.0,1.0,1.0,0.949000,0.948956,0.991298
RF Letter 2 Trial 3,1.0,1.0,1.0,0.945933,0.946380,0.990723
RF Letter 2 Trial 4,1.0,1.0,1.0,0.941867,0.940852,0.989595
RF Letter 2 Trial 5,1.0,1.0,1.0,0.945533,0.945406,0.991432


Train ACC    1.000000
Train F1     1.000000
Train AUC    1.000000
Test ACC     0.945427
Test F1      0.945073
Test AUC     0.990583
dtype: float64


KNN Results


,Train ACC,Train F1,Train AUC,Test ACC,Test F1,Test AUC
KNN Letter 2 Trial 1,1.0,1.0,1.0,0.954467,0.954680,0.990726
KNN Letter 2 Trial 2,1.0,1.0,1.0,0.956467,0.956475,0.989199
KNN Letter 2 Trial 3,1.0,1.0,1.0,0.957600,0.957492,0.990757
KNN Letter 2 Trial 4,1.0,1.0,1.0,0.954600,0.954415,0.990531
KNN Letter 2 Trial 5,1.0,1.0,1.0,0.956067,0.954298,0.991027


Train ACC    1.000000
Train F1     1.000000
Train AUC    1.000000
Test ACC     0.955840
Test F1      0.955472
Test AUC     0.990448
dtype: float64


In [27]:
# need to save the data for ttests
df_log_reg.to_csv("lr_letter2.csv")
df_rf.to_csv("rf_letter2.csv")
df_knn.to_csv("knn_letter2.csv")